# Parte 3

Esse parte do relatorio foca na criacao de um modelo de classificacao, onde sera definidas duas classes para o nivel de servico da plataforma de video. Serao definidas 2 classes, ou seja, um classificador binario, sendo uma classe de `conforms` quando a taxa de video $\geq 18$ e `violates` caso contrario.

1. Model Training - use `Logistic Regression` to train a classifier C with the training set. Provide the coefficients $(\theta_0, ..., \theta_9)$ of your model C. ($\theta_0$ is the offset.)

2. Accuracy of the Classifiers C - Compute the classification error (ERR) on the test set for C. For this, you first compute the confusion matrix, which includes the four numbers True Positives (TP), True Negatives (TN), False Positives (FN), and False Negatives (FN). We define the classification error as ERR = $1 - \frac{TP+TN}{m}$, whereby $m$ is the number of observations in the test set. A true positive is observation that is correctly classified by the classifier as conforming to the SLA; a true negative is an observation that is correctly classified by the classifier as violating the SLA.

3. As a baseline for C, use a naı̈ve method which relies on Y values only, as follows. For each $x \in X$, the naı̈ve classifier predicts a value `True` with probability $p$ and `False` with probability $1 − p$. $p$ is the fraction of $Y$ values that conform with the SLA. Compute $p$ over the training set and the classification error for the naı̈ve classifier over the test set.

4. Build a new classifier by extending extend the linear regression function developed in `Task II` with a check on the output, i.e., the Video Frame Rate. If the frame rate for a given X is above the SLA threshold, then the Y label of the classifier is set to conformance, otherwise to violation. Compute the new classifier over the training set and the classification error for this new classifier over the test set.

5. Formulate your observations and conclusions based on the above work.

In [1]:
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as pp
from sklearn.metrics import confusion_matrix

x_data = pd.read_csv('./data/X.csv')
y_output = pd.read_csv('./data/Y.csv')

# Use 70% data for training and 30% for testing.
up_to = 2520
x_train, x_test, y_train_continuous, y_test_continuous = \
    x_data[:up_to], x_data[up_to:], y_output[:up_to], y_output[up_to:]

def segment_classes(fps):
    return [1 if frame >= 18 else 0 for frame in fps]

y_train = segment_classes(y_train_continuous['DispFrames'])
y_test = segment_classes(y_test_continuous['DispFrames'])

model = linear_model.LogisticRegression()
model.fit(x_train, y_train)

predicted = model.predict(x_test)
metrics = confusion_matrix(y_test, predicted)
print("Matrix de confusao:\n", metrics)

Matrix de confusao:
 [[ 13  83]
 [ 14 970]]


In [5]:
def compute_err(matrix):
    trues, falses = matrix
    return 1 - ((trues[0] + falses[1]) / len(y_test))

print("Model ERR:", compute_err(metrics))

Model ERR: 0.08981481481481479


In [3]:
predicted_prob = model.predict_proba(x_test)
pd.DataFrame(predicted_prob, columns=["1 - p", "p"])

,1 - p,p
0,0.001676,0.998324
1,0.002052,0.997948
2,0.001890,0.998110
3,0.002701,0.997299
4,0.000972,0.999028
...,...,...
1075,0.383426,0.616574
1076,0.266676,0.733324
1077,0.299816,0.700184
1078,0.241755,0.758245


In [8]:
# Now we build a Linear Regression model and train it on the same data.
regression_model = linear_model.LinearRegression()
regression_model.fit(x_train, y_train_continuous['DispFrames'])

# Predict some values.
continuous_pred = regression_model.predict(x_test)
normalized_pred = segment_classes(continuous_pred)

calculated_err = compute_err(confusion_matrix(y_test, normalized_pred))
print("Regressao transformada ERR:", calculated_err)

Regressao transformada ERR: 0.0888888888888889


Diferente do modelo de regressao, esse modelo tenta classificar entre as classes. Pode ser visto que esse modelo tambem age de uma maneira moderada em face de oscilacoes na taxa de quadros, podendo ser visto pela matrix de confusao e os valores de falsos positivos.

O modelo de classificacao criado utilizando o modelo de regressao mostrou um resultado semelhante ao modelo de classificacao criado originalmente. Ambos modelos possuiram o `ERR` com valor de `0.089`, mas esse resultado semelhante pode ter sido pelo fato de existir somente 2 classes, se existissem mais classes de SLA talvez esse valor nao seria tao semelhante e o modelo utilizando regressao alcancaria um resultado pior que a classificacao.